In [1]:
from models import caption_generator

/Users/josepsmachine/miniforge3/envs/ML/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/josepsmachine/miniforge3/envs/ML/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: dlopen(/Users/josepsmachine/miniforge3/envs/ML/lib/python3.10/site-packages/torchvision/image.so, 0x0006): Symbol not found: (__ZN3c106detail19maybe_wrap_dim_slowExxb)
  Referenced from: '/Users/josepsmachine/miniforge3/envs/ML/lib/python3.10/site-packages/torchvision/image.so'
  Expected in: '/Users/josepsmachine/miniforge3/envs/ML/lib/python3.10/site-packages/torch/lib/libc10.dylib'
  warn(f"Failed to load image Python extension: {e}")


In [2]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
gpt2 = GPT2LMHeadModel.from_pretrained('gpt2')

In [3]:
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
resnet18 = models.resnet18(pretrained=True)

/Users/josepsmachine/miniforge3/envs/ML/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/josepsmachine/miniforge3/envs/ML/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [4]:
device = "mps"

In [5]:
model = caption_generator(gpt2,resnet18,tokens_per_img=5,batch_size=2)
model.to(device)

caption_generator(
  (gpt2): GPT2LMHeadModel(
    (transformer): GPT2Model(
      (wte): hacky_embedding(
        (gpt2_emb): Embedding(50257, 768)
      )
      (wpe): Embedding(1024, 768)
      (drop): Dropout(p=0.1, inplace=False)
      (h): ModuleList(
        (0-11): 12 x GPT2Block(
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (attn): GPT2Attention(
            (c_attn): Conv1D()
            (c_proj): Conv1D()
            (attn_dropout): Dropout(p=0.1, inplace=False)
            (resid_dropout): Dropout(p=0.1, inplace=False)
          )
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): GPT2MLP(
            (c_fc): Conv1D()
            (c_proj): Conv1D()
            (act): NewGELUActivation()
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
      )
      (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    )
    (lm_head): Linear(in_features=768, out_features=50257, bi

In [6]:
from dataset import image_caption_dataset

In [7]:
dataset = image_caption_dataset("/Users/josepsmachine/Documents/UNI/DL/dlnn-project_ia-group_10_OLDDD/dataset/Images/","/Users/josepsmachine/Documents/UNI/DL/dlnn-project_ia-group_10_OLDDD/dataset/captions.txt",n_im=100)

100%|██████████| 100/100 [00:00<00:00, 227.52it/s]


In [8]:
sample = dataset.__getitem__(0)
img = sample["x"].to(device)
caption = sample["y_tokenized"].to(device)

In [9]:
from torch.utils.data import DataLoader

In [10]:
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

In [11]:
sample = next(iter(dataloader))
img = sample["x"].to(device)
caption = sample["y_tokenized"].to(device).squeeze(0)

In [12]:
caption_logits = model(img=img,train=True,caption=caption)

In [13]:
import torch.nn.functional as F
import torch.nn as nn
entropy_loss = nn.CrossEntropyLoss()

In [14]:
loss = entropy_loss(F.softmax(caption_logits[0]),caption[0])
loss 

/var/folders/j8/64lnvrmj50q5dv_5dj_ztbz40000gn/T/ipykernel_92365/1536336104.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  loss = entropy_loss(F.softmax(caption_logits[0]),caption[0])
/Users/josepsmachine/miniforge3/envs/ML/lib/python3.10/site-packages/torch/_tensor_str.py:115: UserWarning: MPS: nonzero op is supported natively starting from macOS 13.0. Falling back on CPU. This may have performance implications. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/mps/operations/Indexing.mm:218.)
  nonzero_finite_vals = torch.masked_select(


tensor(10.8233, device='mps:0', grad_fn=<NllLossBackward0>)

In [15]:
loss.backward()

In [16]:
from train import train

In [17]:
train(model,dataloader,epochs=3,lr=0.001)

torch.Size([16])
torch.Size([16, 50257])
epoch: 0 || batch:0 || loss:10.824129104614258
torch.Size([25])
torch.Size([25, 50257])
epoch: 0 || batch:1 || loss:10.78125
torch.Size([24])
torch.Size([24, 50257])
epoch: 0 || batch:2 || loss:10.81361198425293
torch.Size([27])
torch.Size([27, 50257])
epoch: 0 || batch:3 || loss:10.760064125061035
torch.Size([22])
torch.Size([22, 50257])
epoch: 0 || batch:4 || loss:10.732251167297363
torch.Size([16])
torch.Size([16, 50257])


KeyboardInterrupt: 

: 